## 2023 ChemiCloud Task 3 
關鍵字元組的抽取
是否能夠看成兩個子任務 NER + RE (Relation Extraction) 
，以現有checkpoints來完成？
- [ckip tagger ner paper](https://arxiv.org/pdf/1908.11046.pdf) (Att BiLSTM CNN 的數據)
- [Ckip 使用 OntoNotes Entity type list](https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf)

Entity|Description
|---|---|
PERSON|People, including fictional
NORP | Nationalities or religious or political groups
FACILITY| Buildings, airports, highways, bridges, etc.
ORGANIZATION | Companies, agencies, institutions, etc.
GPE | Countries, cities, states
LOCATION | Non-GPE locations, mountain ranges, bodies of water
PRODUCT | Vehicles, weapons, foods, etc. (Not services)
EVENT | Named hurricanes, battles, wars, sports events, etc.
WORK OF ART| Titles of books, songs, etc.
LAW | Named documents made into laws 
LANGUAGE| Any named language
DATE | Absolute or relative dates or periods
TIME | Times smaller than a day
PERCENT| Percentage (including “%”)
MONEY | Monetary values, including unit
QUANTITY| Measurements, as of weight or distance
ORDINAL| “first”, “second”
CARDINAL| Numerals that do not fall under another type

- ckip tagger 
- link: https://github.com/ckiplab/ckiptagger 

|F1 score| OntoNotes 5.0 |WNUT 
|---|---|---|
|ckip tagger NER |88.4%+-0.18|42.26%+-0.82

- ckip transformers 
- link: https://github.com/ckiplab/ckip-transformers 

|F1 score| OntoNotes 5.0 |
|---|---|
|ckip transformers NER | 81.17% # bert-base-chinese 


In [6]:
import os 
import pandas as pd 
root = '/home/nanaeilish/projects/2023-chemicloud/'
data_dir = os.path.join(root, 'data')
filename = '食品安全_ws.csvpkl'
df = pd.read_pickle(os.path.join(data_dir, filename))


In [9]:
titlecol = '新聞標題'
textcol = '新聞內容' 
texts = df[textcol].tolist()
titles = df[titlecol].tolist() 

## Ckip Transformers

In [22]:
from ckip_transformers.nlp import CkipPosTagger, CkipNerChunker 

ner_driver = CkipNerChunker(model="bert-base") 

df['ner_ckiptrans_text'] = ner_driver(texts) 
df['ner_ckiptrans_title'] = ner_driver(titles)

Inference: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


In [21]:
from termcolor import colored 
from typing import List, Dict
from ckip_transformers.nlp.util import NerToken 


colormap = {
    'PERSON': 'red',
    'FOOD': 'blue',
    'PRODUCT': 'blue',
    'CHEMICAL': 'magenta',      
    'ORG': 'green',
    'DATE': 'yellow', 
}


def color_ner_text(text: str, 
                   nertokens: List[NerToken], 
                   colormap: Dict = colormap,)-> None:
    """ Color the text with the NER tokens. 
        Args:
            text (str): The text to be colored.
            nertokens (List[NerToken]): The NER tokens.
                It can be simulated with an object with the following attributes:
                    ner (str): The NER tag.
                    idx (Tuple[int, int]): The start and end index of the token. 
    """
    nertokens = sorted(nertokens, key=lambda x: x.idx[0], reverse = True)
    for nertoken in nertokens:
        b, e = nertoken.idx 
        color = colormap.get(nertoken.ner, 'cyan')
        typed_text = text[b:e] + f'_{nertoken.ner}' + ' '
        span = colored(typed_text, color) 
        text = text[:b] + span + text[e:]
    print(text)


In [6]:
news_id = 13 
df.iloc[news_id][titlecol]

'快訊 哈根達斯 香草冰淇淋 再驗出禁用農藥 香港喊停售'

In [32]:
print('==== ckip transformers ===')
text = df.iloc[news_id][textcol] 
nertokens = df.iloc[news_id]['ner_text']
color_ner_text(text = text, nertokens = nertokens)

==== ckip transformers ===
▲哈根達斯_PERSON 香草冰淇淋再被驗出致癌物環氧乙烷。（圖／食藥署提供）
記者鄒鎮宇_PERSON ／綜合報導
美國_GPE 知名冰淇淋品牌「Häagen-Dazs（哈根達斯）」日前進口台灣_GPE 的「香草冰淇淋」驗出禁用農藥，被邊境攔下依規定銷毀。沒想到，香港食物環境衞生署_ORG 食物安全中心10日_DATE 發出公告，哈根達斯_PERSON 的香草冰淇淋被驗出歐盟_ORG 禁用的農藥環氧乙烷，因此要求業界停止使用或出售。
據《東網》報導，香港食物環境衞生署_ORG 食物安全中心發現，哈根達斯75毫升_QUANTITY 、100毫升_QUANTITY 、473毫升_QUANTITY 、9.46公升_QUANTITY 裝的香草冰淇淋被驗出農藥環氧乙烷，因此立刻跟進口商溝通，並通知業界停止使用或出售，後續將進行調查。
哈根達斯_PERSON 6月21日_DATE 時也在香港_GPE 被驗出有環氧乙烷，當時香港_GPE 哈根達斯_PERSON 致歉，並停售、撤回商品，豈料本月10日_DATE 又再被驗出含有環氧乙烷。
據悉，哈根達斯_PERSON 6月21日_DATE 時進口台灣_GPE 的香草冰淇淋也驗出環氧乙烷，在邊境被攔下1164盒、5471.34公斤_QUANTITY 的產品，因不符合食品安全衛生管理法第15_ORDINAL 條有關「農藥殘留容許量標準」規定，須依規定退運或銷毀。
食藥署_ORG 北區管理中心簡任技正吳宗熹_PERSON 過去受訪時指出，環氧乙烷為農藥一種，具致癌風險，依目前規定不在食品中檢出，國內也沒有核准作為農藥使用，因此進口產品也不得檢出。
►按這訂閱Podcast《小編沒收工_WORK_OF_ART 》每天熱門話題聊不完


## Ckip Tagger 

In [33]:
# (1, 10, 'PRODUCT', '哈根達斯香草冰淇淋')
from typing import Tuple 
from easydict import EasyDict as edict 

def quad2nertoken(quad: Tuple):
    """ Convert a quad to a NER token.
    
        Args:
            quad (Tuple): The quad to be converted.
                It should be (start, end, ner, text).
        Returns:
            A NER token-like object. Simulated with edict() 
    """
    nertok = edict()
    nertok.idx = (quad[0], quad[1])
    nertok.ner = quad[2]
    nertok.text = quad[3]
    return nertok 


In [26]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ckiptagger_path = "./ckiptagger"
data_utils.download_data_gdown(ckiptagger_path) # gdrive-ckip
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



ckipdata_path = f'{ckiptagger_path}/data'
ws = WS(ckipdata_path , disable_cuda=False)
pos = POS(ckipdata_path, disable_cuda=False)
ner = NER(ckipdata_path, disable_cuda=False)


# titles 

word_sentence_list = ws(
    titles,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced
)
pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list) 
df['ner_ckiptagger_title'] = [list(map(quad2nertoken, entity)) for entity in entity_sentence_list]

# texts 
word_sentence_list = ws(
    texts,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced
)
pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list) 

df['ner_ckiptagger_text'] = [list(map(quad2nertoken, entity)) for entity in entity_sentence_list] 



Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /home/nanaeilish/projects/2023-chemicloud/ckiptagger/data.zip
100%|██████████| 1.88G/1.88G [00:37<00:00, 49.6MB/s]
/home/nanaeilish/micromamba/envs/chemicloud/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2023-02-26 14:36:48.613837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-26 14:36:48.614050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared ob

In [44]:

import random 
news_id = random.randint(0, len(df))
print('==== ckip tagger ===')
print(news_id)
text = df.iloc[news_id][textcol] 
nertokens = df.iloc[news_id]['ner_ckiptagger_text']
color_ner_text(text = text, nertokens = nertokens)

==== ckip tagger ===
1
▲日本_GPE 福島_GPE 核災五_CARDINAL 縣市食品有條件輸入台灣_GPE 。（示意圖／記者林敬旻_PERSON 攝）

文／食力foodNEXT
採訪／李依文
撰文／黃宜稜_PERSON 、李依文_NORP 
2011年3月11日_DATE 日本_GPE 東部發生地震規模9.0_CARDINAL 的大地震，為日本_GPE 有紀錄以來規模最大的地震，並伴隨著海嘯與餘震引發一連串大規模的災害，其中福島第一核電廠事故_EVENT 更因此造成設備損害、爐心熔毀、輻射釋放等災害，是為自1986年_DATE 車諾比核電廠事件_EVENT 後最嚴重的核災事件。由於輻射外洩恐污染臨近區域所種植、製造的食品，因此各國政府與日本_NORP 官方開始制定一連串的政策與檢驗標準。
2011年3月_DATE 福島_GPE 5_CARDINAL 縣食品暫停輸台 9_CARDINAL 大類每日逐批查驗
台灣_GPE 自2011年3月25日_DATE 暫停受理日本_GPE 福島縣與鄰近的茨城縣_GPE 、櫪木縣_GPE 、群馬縣_GPE 、千葉縣_GPE 等共5_CARDINAL 個縣市生產製造的食品輸入，而對於日本_GPE 福島_GPE 5_CARDINAL 縣以外地區的食品輸入台灣_GPE ，則針對9_CARDINAL 大類食品包含生鮮冷藏蔬果、冷凍蔬果、活生鮮冷藏水產品、冷凍水產品、乳製品、嬰幼兒食品、礦泉水或飲水、海草類、茶葉等實施逐批查驗，並送請行政院_ORG 原子能委員會檢測其輻射量，以加馬能譜分析碘-13_CARDINAL 1、銫-134、銫-137_PRODUCT 等人工核種，每個工作天於食藥署_ORG 「日本食品管理工作專區_ORG 」公告結果，截自2016年11月23日_DATE 累積檢驗數已達9萬4438_CARDINAL 件，其中雖有216_CARDINAL 件被檢出微量輻射，但檢出值並未超過我國與日本_GPE 所制定的「原子塵或放射能污染安全容許量標準」。
2013年7月_DATE 輻射水再度外洩 福島_GPE 5_CARDINAL 縣持續暫停輸台
2013年7月初_DATE ，福島_GPE 第一核電廠再度發生輻射水外洩事件，而日本東京電力公司_ORG 卻直到8月31日_DATE 才出面證實，因此同年9月6日_D

In [ ]:
df = df.drop(columns = ['ner_text', 'ner_title'])
df.to_pickle(os.path.join(data_dir, filename))

## PaddleNLP: UIE 

In [34]:
from typing import Union
import opencc
def t2sconvert(texts: Union[List, Dict, str])-> Union[List, Dict, str]:

    converter = opencc.OpenCC('t2s.json')
    if isinstance(texts, str):
        new_texts = converter.convert(texts)
    elif isinstance(texts, list):
        new_texts = []
        for i in range(len(texts)):
            new_texts.append(converter.convert(texts[i]))
    elif isinstance(texts, dict):
        new_texts = {}
        for key, value in texts.items():
            key = converter.convert(key)
            new_texts[key] = t2sconvert(value)
    assert len(texts) == len(new_texts)
    del texts
    return new_texts

In [35]:
schema = {"機構": [], "化學物質": ["治癌物", "農藥"], "產品": [], "時間": []} 

In [40]:
from typing import Dict, List
from easydict import EasyDict as edict
keymap = {
    '機構': 'ORG',
    '化學物質': 'CHEMICAL',
    '產品': 'PRODUCT',
    '時間': 'TIME',
}
def uie2nertokens(uie: Dict)-> List:
    """ Convert uie output to a list of NER tokens. 
        Args:
            uie (Dict): uie output, a dict with
                the schema keys and a list of tuples as extracted tokens. 
            Examples: 
            {'产品': [{'end': 83,
                    'probability': 0.32381882254490435,
                    'start': 79,
                    'text': '三苯醋锡'},
                    {'end': 61,
                    'probability': 0.4285592181817037,
                    'start': 56,
                    'text': '氯化三苯锡'}], 
            '机构': [...]} 
        Returns:
            List: a list of ner tokens simulated with edict()  
    """
    nertoks = [] 
    for k in schema:
        simpk = t2sconvert(k)
        v = uie.get(simpk, []) 
        entity_name = keymap.get(k, k)
        for entity in v: 
            nertok = edict()
            nertok.idx = (entity['start'], entity['end'])
            nertok.ner = entity_name
            nertoks.append(nertok)
    return nertoks 

In [41]:

# colormap = {
#     'PERSON': 'red',
#     'FOOD': 'blue',
#     'PRODUCT': 'blue',
#     'CHEMICAL': 'magenta',      
#     'ORG': 'green',
#     'DATE': 'yellow', 
# }


In [52]:
import random 
# news_id = random.randint(0, len(df))
# print(news_id)
news_id  = 13 

In [53]:
filename = '食品安全_uie.csvpkl'
df = pd.read_pickle(os.path.join(data_dir, filename)) 
text = df.iloc[news_id][textcol] 
uie_output = df.iloc[news_id]['uie_base'][0]
nertokens = uie2nertokens(uie_output)
color_ner_text(text = text, nertokens = nertokens)

▲哈根達斯_ORG 香草冰淇淋再被驗出致癌物環氧乙烷_CHEMICAL 。（圖／食藥署_ORG 提供）
記者鄒鎮宇／綜合報導
美國知名冰淇淋品牌「Häagen-Dazs（哈根達斯）」日前進口台灣的「香草冰淇淋」_PRODUCT 驗出禁用農藥，被邊境攔下依規定銷毀。沒想到，香港食物環境衞生署食物安全中心_ORG 10日發出公告，哈根達斯_ORG 的香草冰淇淋被驗出歐盟_ORG 禁用的農藥環氧乙烷_CHEMICAL ，因此要求業界停止使用或出售。
據《東網》報導，香港食物環境衞生署食物安全中心_ORG 發現，哈根達斯_ORG 75毫升、100毫升、473毫升、9.46公升裝的香草冰淇淋被驗出農藥環氧乙烷_CHEMICAL ，因此立刻跟進口商溝通，並通知業界停止使用或出售，後續將進行調查。
哈根達斯_ORG 6月21日時也在香港被驗出有環氧乙烷_CHEMICAL ，當時香港哈根達斯_ORG 致歉，並停售、撤回商品，豈料本月10日又再被驗出含有環氧乙烷_CHEMICAL 。
據悉，哈根達斯_ORG 6月21日時進口台灣的香草冰淇淋也驗出環氧乙烷，在邊境被攔下1164盒、5471.34公斤的產品，因不符合食品安全衛生管理法第15條有關「農藥殘留容許量標準」規定，須依規定退運或銷毀。
食藥署北區管理中心_ORG 簡任技正吳宗熹過去受訪時指出，環氧乙烷為農藥一種，具致癌風險，依目前規定不在食品中檢出，國內也沒有核准作為農藥使用，因此進口產品也不得檢出。
►按這訂閱Podcast《小編沒收工》每天熱門話題聊不完



## Droidtown:Articut 

#### References 
https://www.jsjkx.com/EN/10.11896/jsjkx.200800181 -> food safety news 
#### Issue: chemical substance: ckip-transformers 環氧乙烷 抓不出來 (Chemical NER) 
-> 求助易庭的 dictionary tree  
-> 卓騰 Articut Chemical NER (Python API available): https://blog.droidtown.co/post/643573663484067840/chemical 


#### Issue: ckip-transformers 香草冰淇淋抓不出來（food NER)  
-> 卓騰 Articut Food NER: https://api.droidtown.co/ArticutAPI/document/#ArticutAPI
 
- foodBERT
  https://github.com/chambliss/foodbert -> English only  
- Chinese ner dataset: https://zhuanlan.zhihu.com/p/529541521
    - ner food related dataset: 萬創杯中醫相關命名實體辨識資料集
      https://aiqianji.com/openoker/Chinese-DeepNER-Pytorch 
    - 其中有食物的 entity annotation，但是感覺比較偏向原型食物：
```
食物(FOOD):指能够满足机体正常生理和生化能量需求，并能延续正常寿命的物质。对人体而言，能够满足人的正常生活活动需求并利于寿命延长的物质称之为食物。例子：苹果、茶、木耳、萝卜
食物分组(FOOD_GROUP): 中医中饮食养生中，将食物分为寒热温凉四性，同时中医药禁忌中对于具有某类共同属性食物的统称，记为食物分组。例子：油腻食物、辛辣食物、凉性食物
```
- English food dataset: 
- TASTEset (Recipe Dataset and Food Entities Recoginition Benchmark): https://github.com/taisti/tasteset 
```
ingredients,ingredients_entities
"5 ounces rum
4 ounces triple sec
3 ounces Tia Maria
20 ounces orange juice
","[{""start"": 0, ""end"": 1, ""type"": ""QUANTITY"", ""entity"": ""5""},{""start"": 2, ""end"": 8, ""type"": ""UNIT"", ""entity"": ""ounces""},{""start"": 9, ""end"": 12, ""type"": ""FOOD"", ""entity"": ""rum""},{""start"": 13, ""end"": 14, ""type"": ""QUANTITY"", ""entity"": ""4""},{""start"": 15, ""end"": 21, ""type"": ""UNIT"", ""entity"": ""ounces""},{""start"": 22, ""end"": 32, ""type"": ""FOOD"", ""entity"": ""triple sec""},{""start"": 33, ""end"": 34, ""type"": ""QUANTITY"", ""entity"": ""3""},{""start"": 35, ""end"": 41, ""type"": ""UNIT"", ""entity"": ""ounces""},{""start"": 42, ""end"": 51, ""type"": ""FOOD"", ""entity"": ""Tia Maria""},{""start"": 52, ""end"": 54, ""type"": ""QUANTITY"", ""entity"": ""20""},{""start"": 55, ""end"": 61, ""type"": ""UNIT"", ""entity"": ""ounces""},{""start"": 62, ""end"": 74, ""type"": ""FOOD"", ""entity"": ""orange juice""}]"
``` 
- FoodBase corpus 
https://academic.oup.com/database/article/doi/10.1093/database/baz121/5611291?login=true


In [79]:
from ArticutAPI import Articut

articut = Articut() 
# default: public quota mode 
# 每小時更新 2000 字
news_id = 13 
inputSTR = df.iloc[news_id][textcol]

result = articut.parse(inputSTR, chemicalBOOL = True)
# food_ner_tokens = articut.NER.getFood(result)

In [51]:
from easydict import EasyDict as edict 

def format_ner_tokens(ner_tokens, type = 'FOOD'):
    raw = [edict({'idx': (t[0][1] - len(t[0][2]), t[0][1]), 
                   'text': t[0][2],
                   'ner': type}) for t in ner_tokens if len(t) > 0]
    return raw


In [62]:

chemical_ner_tokens = articut.getChemicalLIST(result, indexWithPOS=True) 


In [ ]:
chemical_ner_tokens

In [78]:

# get correct indices (navigating into Droidtown's code)
""" # ARTICUTAPI/ToolKit/toolkits.py
    def getChemicalLIST(self, parseResultDICT, indexWithPOS=True):
        '''
        取出斷詞結果中的 KNOWLEDGE_chemical。
        每個句子內的 KNOWLEDGE_chemical 為一個 list.
        '''
        if "result_pos" in parseResultDICT:
            pass
        else:
            return None
        chemicalLIST = []
        for p in parseResultDICT["result_pos"]:
            if len(p) > 1:
                chemicalLIST.append([(c.start(), c.end(), c.group(0))
                                     for c in list(self.chemicalPat.finditer(p))])
            else:
                chemicalLIST.append([])
        if not indexWithPOS:
            chemicalLIST = self._segIndexConverter(parseResultDICT, chemicalLIST)
        return chemicalLIST
"""
nertype = 'CHEMICAL'

result_pos = result['result_pos']
paragraph = ''
for p, t in zip(result_pos, chemical_ner_tokens):
    if len(t) > 0:
        t.sort(key = lambda x: x[-1], reverse = True)
        for (b, e, token_text) in t:
            typed_text = p[b:e] + f'_{nertype}' + ' '
            color = colormap.get(nertype, 'cyan')
            span = colored(typed_text, color) 
            p = p[:b] + span + p[e:]
            # print(p)
            paragraph += p
    else: 
        paragraph += p if isinstance(p, str) else ' '.join(p)
# print(paragraph)
# format_chemi_tokens = format_ner_tokens(chemical_ner_tokens , type = 'CHEMICAL')
# format_chemi_tokens 


In [77]:

#  clean all the <tag></tag> 
import re 
CLEANR = re.compile('<.*?>') 

def cleantags(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

clean_paragraph = cleantags(paragraph)
print(clean_paragraph)

▲哈根達斯香草冰淇淋再被驗出致癌物環氧乙烷_CHEMICAL 。（圖／食藥署提供）
記者鄒鎮宇／綜合報導
美國知名冰淇淋品牌「Häagen-Dazs（哈根達斯）」日前進口台灣的「香草冰淇淋」驗出禁用農藥，被邊境攔下依規定銷毀。沒想到，香港食物環境衞生署食物安全中心10日發出公告，哈根達斯的香草冰淇淋被驗出歐盟禁用的農藥環氧乙烷_CHEMICAL ，因此要求業界停止使用或出售。
據《東網》報導，香港食物環境衞生署食物安全中心發現，哈根達斯75毫升、100毫升、473毫升、9.46公升裝的香草冰淇淋被驗出農藥環氧乙烷_CHEMICAL ，因此立刻跟進口商溝通，並通知業界停止使用或出售，後續將進行調查。
哈根達斯6月21日時也在香港被驗出有環氧乙烷_CHEMICAL ，當時香港哈根達斯致歉，並停售、撤回商品，豈料本月10日又再被驗出含有環氧乙烷_CHEMICAL 。
據悉，哈根達斯6月21日時進口台灣的香草冰淇淋也驗出環氧乙烷_CHEMICAL ，在邊境被攔下1164盒、5471.34公斤的產品，因不符合食品安全衛生管理法第15條有關「農藥殘留容許量標準」規定，須依規定退運或銷毀。
食藥署北區管理中心簡任技正吳宗熹過去受訪時指出，環氧乙烷_CHEMICAL 為農藥一種，具致癌風險，依目前規定不在食品中檢出，國內也沒有核准作為農藥使用，因此進口產品也不得檢出。
►按這訂閱Podcast《小編沒收工》每天熱門話題聊不完
